<a href="https://colab.research.google.com/github/vidhyagowda-232/Microsoft-Research-Paraphrase-Corpus/blob/main/BERT_MRPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0"; 

In [3]:
import pandas as pd
import csv
TRAIN = '/content/msr_paraphrase_train.txt'
TEST = '/content/msr_paraphrase_test.txt'
train_df = pd.read_csv(TRAIN, delimiter='\t', quoting=csv.QUOTE_NONE)
test_df = pd.read_csv(TEST, delimiter='\t', quoting=csv.QUOTE_NONE)
x_train = train_df[['#1 String', '#2 String']].values
y_train = train_df['Quality'].values
x_test = test_df[['#1 String', '#2 String']].values
y_test = test_df['Quality'].values


# IMPORTANT: data format for sentence pair classification is list of tuples of form (str, str)
x_train = list(map(tuple, x_train))
x_test = list(map(tuple, x_test))

In [4]:
print(x_train[0])
print(y_train[0])

('Amrozi accused his brother, whom he called "the witness", of deliberately distorting his evidence.', 'Referring to him as only "the witness", Amrozi accused his brother of deliberately distorting his evidence.')
1


Build and Train a BERT Model

In [5]:
!pip install ktrain
import ktrain
from ktrain import text
MODEL_NAME = 'bert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=128, class_names=['not paraphrase', 'paraphrase'])
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=32) # lower bs if OOM occurs
learner.fit_onecycle(5e-5, 3)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
preprocessing train...
language: en
sentence pairs detected


Is Multi-Label? False
preprocessing test...
language: en
sentence pairs detected




begin training using onecycle policy with max lr of 5e-05...
Epoch 1/3
128/128 [==============================] - 6419s 50s/step - loss: 0.5982 - accuracy: 0.6793 - val_loss: 0.4951 - val_accuracy: 0.7641
Epoch 2/3
128/128 [==============================] - 6262s 49s/step - loss: 0.4133 - accuracy: 0.8111 - val_loss: 0.3875 - val_accuracy: 0.8400
Epoch 3/3
128/128 [==============================] - 6160s 48s/step - loss: 0.1773 - accuracy: 0.9369 - val_loss: 0.4872 - val_accuracy: 0.8313


Make Predictions

In [6]:
predictor = ktrain.get_predictor(learner.model, t)

In [7]:
y_test[:5]

array([1, 1, 1, 0, 0])

In [8]:
positive = x_test[0]
negative = x_test[4]

In [9]:
print('Valid Paraphrase:\n%s' %(positive,))

Valid Paraphrase:
("PCCW's chief operating officer, Mike Butcher, and Alex Arena, the chief financial officer, will report directly to Mr So.", 'Current Chief Operating Officer Mike Butcher and Group Chief Financial Officer Alex Arena will report to So.')


In [10]:
print('Invalid Paraphrase:\n%s' %(negative,))

Invalid Paraphrase:
("The company didn't detail the costs of the replacement and repairs.", 'But company officials expect the costs of the replacement work to run into the millions of dollars.')


In [11]:
predictor.predict(positive)

'paraphrase'

In [12]:
predictor.predict(negative)

'not paraphrase'

In [13]:
predictor.predict([positive, negative])

['paraphrase', 'not paraphrase']

In [14]:
predictor.save('/tmp/mrpc_model')

In [15]:
p = ktrain.load_predictor('/tmp/mrpc_model')

In [16]:
p.predict(positive)

'paraphrase'